In [6]:
from datasets import load_dataset

dataset = load_dataset("amazon_polarity")
dataset

In [7]:
dataset.set_format("pandas")

In [8]:
from arize.pandas.embeddings import EmbeddingGenerator, UseCases
model_name = "arize-ai/distilbert_reviews_with_language_drift"
generator = EmbeddingGenerator.from_use_case(
    use_case=UseCases.NLP.SEQUENCE_CLASSIFICATION,
    model_name=model_name,
)


  arize.utils.logging | WARNING | No available GPU has been detected. The use of GPU acceleration is strongly recommended. You can check for GPU availability by running `torch.cuda.is_available()`
  arize.utils.logging | INFO | Downloading pre-trained model 'arize-ai/distilbert_reviews_with_language_drift'
  arize.utils.logging | INFO | Downloading tokenizer for 'arize-ai/distilbert_reviews_with_language_drift'


In [12]:
train_df = dataset["train"][:1000]
test_df = dataset["test"][:100]

In [13]:
train_df["content_vector"] = generator.generate_embeddings(text_col=train_df["content"])
test_df["content_vector"] = generator.generate_embeddings(text_col=test_df["content"])

  arize.utils.logging | INFO | Generating embedding vectors


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

  arize.utils.logging | INFO | Generating embedding vectors


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [14]:
import phoenix as px


schema = px.Schema(
    actual_label_column_name="label",
    embedding_feature_column_names={
        "text_embedding": px.EmbeddingColumnNames(
            vector_column_name="content_vector", raw_data_column_name="content"
        ),
    },
)


prod_ds = px.Dataset(dataframe=test_df, schema=schema, name="testing")
train_ds = px.Dataset(dataframe=train_df, schema=schema, name="training")



In [15]:
session = px.launch_app(primary=prod_ds, reference=train_ds)


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [16]:
px.active_session().view()

📺 Opening a view to the Phoenix app. The app is running at http://localhost:6006/
